In [ ]:
import numpy as np
from scipy.stats import norm 
import matplotlib.pyplot as plt
import matplotlib.animation as animation

: 

In [ ]:
sigma_u = 1
vp = 5
sigma_p = 1
infer_time = 2000
epoch = 100
dt = 0.01
phi = list(range(infer_time))
ep = list(range(infer_time))
eu = list(range(infer_time))

In [ ]:
input_size, hidden_size, output_size = 2, 30, 1
learning_rate = 0.01
np.random.rand(0)
W1 = np.random.randn(input_size, hidden_size)  / np.sqrt(input_size) * np.sqrt(2)
W2 = np.random.randn(hidden_size, output_size)  / np.sqrt(hidden_size) * np.sqrt(2)

# 順伝播
def forward(phi):
    global W1
    global W2
    H = np.tanh(np.dot(phi, W1))
    U = np.dot(H, W2)
    return U, H

# 逆伝播
def backward(phi, H, U, D): # 予測値、　隠れ層、　出力層、　教師
    global W1
    global W2
    error_output = U - D # 出力層の誤差
    error_hidden = (1 - H * H) * np.dot(W2, error_output) # 隠れ層の誤差 # 隠れ層が複数ノードだったらaxis=1
    W1 -= np.dot(phi.reshape(len(phi), 1), error_hidden.reshape(1, len(error_hidden))) * learning_rate
    W2 -= np.dot(H.reshape(len(H), 1), error_output.reshape(1, len(error_output))) * learning_rate

def g(phi, u):
    phi = np.array([phi])
    phi = np.insert(phi, 0, 1)
    # 順伝播＆逆伝播
    U, H =forward(phi)
    backward(phi, H, U, u) 
    return U, H

def g_dash(H):    
    tanh_dash = (1 - H * H)
    U_dash = np.dot(tanh_dash * W1, W2)
    return U_dash[1,0]

In [ ]:
# v = [np.random.randint(1, 5) for i in range(epoch)]
v = 3
u = v * v

In [ ]:
for i in range(epoch):
#     u = v[i] * v[i]
    phi[0] = vp
    ep[0] = 0
    eu[0] = 0

    U, H = g(phi[0], u)
    U_dash = g_dash(H)

    # 推測
    for j in range(1, infer_time):
        phi[j] = phi[j-1] + dt * (-ep[j-1] + eu[j-1] * U_dash)
        ep[j] = ep[j-1] + dt * (phi[j-1] - vp - sigma_p * ep[j-1])
        eu[j] = eu[j-1] + dt * (u - U - sigma_u * eu[j-1])

    # 注意
    vp = vp + ep[-1] * dt
    sigma_p = max(sigma_p + 1/2 * (ep[-1] ** 2 - 1/sigma_p) * dt, 1)
    sigma_u = max(sigma_u + 1/2 * (eu[-1] ** 2 - 1/sigma_u) * dt, 1)

    if i == 0 or i == epoch - 1:
        plt.scatter(range(infer_time), phi)
        plt.xlabel("epoch")
        plt.ylabel("φ")
        plt.plot()
        plt.grid()
        plt.show()

#         print("v:", v[i])
        print("v:", v)
        print("phi:", phi[-1])
        print("u:", u)
        print("U:", U)